# DBA Autotune
#### Method

Using Kevin Xu's beam tracking software, we can tune in the DBA automatically according to the rules set forth for the DBA tuning process to ensure a focused and tight beam by the time we reach the cavity. Some of the benefits here will be on the fly tuning so that we can check the DBA setup. We could also do this at different emissions so that we have different conditions in which we are tuning on. This file will be an executable so that it will not run in the background.

Method to follow:

    - This is going to be tough but hear me out.
    - We will require the beam to be on the aperture. Then we are going to loop through to check the following conditions
    - When toggling solenoid two, the beam does not move, the beam should simply rotate slightly, there is something to be said of the rotation but what that is I am not sure.

In [1]:
def Magnet_Calculations(Energy):
    
    # Here we have the required calculated amperage per kG for each magnet type. The ratios below are a G ratio, not an Amperage ratio
    
    Sol_1_ApkG = 1
    Sol_2_ApKG = 1
    DP1_ApkG = 1
    DP2_ApkG = 1
    Sol_3_ApkG = 1
    
    #### Now establish the strengths assuming linear field strength with amperage
    #### We are going to use the dipoles as our baseline, since the solenoids should scale with these values
    
    # --- Ratios
    
    DP_Guass_Calc = 200
    
    DP1_Calc = 1
    DP2_Calc = 1
    Sol_1_Calc = -1.6598
    Sol_2_Calc = -1.9319
    Sol_3_Calc = 1.079
    
    print("The calculated values for the magnets are the following:\n\n" \
          "Dipole 1: {:.3f} Amps\n"
          "Dipole 2: {:.3f} Amps\n"
          "Solenoid 1: {:.3f} Amps\n"
          "Solenoid 2: {:.3f} Amps\n"
          "Solenoid 3: {:.3f} Amps".format(DP1_Calc, DP2_Calc, Sol_1_Calc, Sol_2_Calc, Sol_3_Calc) 
         )
    
    return DP1_Calc, DP2_Calc, Sol_1_Calc, Sol_2_Calc, Sol_3_Calc

Magnet_Calculations(115)

The calculated values for the magnets are the following:

Dipole 1: 1.000 Amps
Dipole 2: 1.000 Amps
Solenoid 1: -1.660 Amps
Solenoid 2: -1.932 Amps
Solenoid 3: 1.079 Amps


(1, 1, -1.6598, -1.9319, 1.079)

In [3]:
def Injector_Energy_Deg(Angle_of_Rotation, Amps):
    '''
    Inputs:
        - Angle_of_Rotation: Angle, in degrees, of the observed rotation
        - Amps: Amps of solenoid 2 setting for the observed rotation
    '''
    
    # Defining our constants
    
    Conversion_Factor = 1000
    M_Electron_keV = 510.9989461
    Speed_of_Light = 299792458
    
    # Observed constants
    Solenoid_2_Tesla_per_Amps = 0.01036
    Effective_Length = 0.069
    
    Term_1 = (0.5)*(Speed_of_Light)/(Conversion_Factor)/(np.deg2rad(Angle_of_Rotation))*(Solenoid_2_Tesla_per_Amps)*(Amps)*(Effective_Length)
    Term_2 = M_Electron_keV
    
    Calculated_Energy = np.sqrt(Term_1**2 + Term_2**2) - M_Electron_keV
    
    print("Sol 2 {0:.3f} Amps has Beam Energy = {1:.2f} keV".format(Amps,Calculated_Energy))
    return Calculated_Energy

In [4]:
def Toggle(Client, Tag_Number):
    '''
    -----#########-----
       NEEDS TESTING
    -----#########-----
    
    Inputs:
        - Client: See Master
        - Tag_Number: Binary Tag number that we will be toggling
    
    Function:
        If the Tag that we read is off (0) then it will immediately write (1) to the register
        Otherwise, if (1), then it will immediately write (0).
        
    Warning: If a value is non-binary, there will be a 1 Written to that tag immediately.
        i.e. proceed with caution
    
    '''
    
    
    M.Write(Client, Tag_Number, (M.Read(Client,Tag_Number) == False, Bool = True), Bool = True)
    
    return

## Process:

#### Establish Beam:
    - Human operator here:
        - Use solenoid 1 and Dipole 1 to put beam onto the selection aperture
        - Keep solenoid 2 off
#### Center Through Solenoid 1:
    - Move Window Frame 1 and Window Frame 2 Until the following conditions are met:
        - When Toggling Solenoid 1, the beam does not move
        - When Scanning Solenoid 1, the beam does not move
    - Place the beam in the center of the aperture using Dipole 1
    - Set solenoid 1 to minimize the beam at the edge of the aperture
#### Set Solenoid 2 to focus angle:
    - Take the start angle of the beam
    - Adjust Solenoid 2 until we reach an angle of 41 degrees off axis
#### Adjust Solenoid 1 to focus on the aperture:
    - Here we are looking to reduce the head of the beam to as small as possible. This can be done in the following ways:
        - Potentially a FWHM Calculation of the beam head at it's brightest spot
        - The length of the beam head, using pixel count
        - This could also be left out and done manually
#### Set Dipole 2:
    - Solenoids 1 and 2, and Dipole 1 is fixed
    - Set Ddipole 2 to the same value as Dipole 1

#### (Optional) Adjust Solenoid 3 to the calculated ratio of magnetic field strength.
    - Calculate the required B-Field for focusing condition to be met
    - Ramp the magnet there, this will just act as a starting point

In [18]:
def Solenoid_1_Check(Client, Solenoid_Tag, Solenoid_Toggle_Tag, Optimize = True):
    
    Sol_1_Start = M.Read(Client, Solenoid_Tag)
    Sol_1_Toggle_Status = M.Read(Solenoid_Toggle_Tag)
    
    #################
    #---- Here I will add in the ability to read the vector from img processing
    #-------- This needs A LOT of work still
    #################
    
    #Check if it moves when toggling
    
    Start_Position = False
    
    Toggle(Client, Solenoid_Toggle_Tag)
    
    Toggled_Position = False
    
    if Toggled_Position is not Start_Position:
        
        return False
        
    else: 
        Data = M.Scan_Two_Way(Client, Solenoid_Tag, Sol_1_Start + 1)
        
        if Data is not True:
            return False
            
            
    #Check in Data where the beam is the smallest
    
    Focused_Beam_Spot = False
    
    if Optimize == True:
        M.Ramp_One_Way(Client, Focused_Beam_Spot)
    
    return True

def Solenoid_Image_Sweep(Client, Camera, Solenoid_Tag, Bottom_Amps, Top_Amps, N_each_way = 2):
    
    Camera = 0 #Stuff you would define from earlier and input into this function
    exposure = 1/3
    gain = 480
    
    Sol_Start = M.Read(Client, Solenoid_Tag)
    
    start_time =  datetime.today().strftime('%y%m%d_%H%M%S')
    filename = "{}_{:.3f}A.bmp".format(start_time, Sol_Start)
    
    M.snap(Camera, exposure, gain, filename) #Takes the starting, focused image
    
    for i in range(1,N_each_way+1):
        '''
        This loop takes N_each_way images as the solenoid overfocuses the beam. The images
            are equally spaced up through Top_Amps
        '''
        temp_position = Sol_Start + (Top_Amps - Sol_Start)*(i/N_each_way)
        
        M.Write(Client, Solenoid_Tag, temp_position)
        
        time.sleep(exposure * 2)
        
        M.snap(Camera, exposure, gain, "{}_{:.3f}A.bmp".format(start_time, temp_position))
        
    M.Write(Client, Solenoid_Tag, Sol_Start) #Write back to start
    
    for i in range(1,N_each_way+1):
        
        temp_position = Sol_Start - (Bottom - Sol_Start)*(i/N_each_way)
        
        M.Write(Client, Solenoid_Tag, temp_position)
        
        time.sleep(exposure * 2)
        
        M.snap(Camera, exposure, gain, "{}_{:.3f}A.bmp".format(start_time, temp_position))
        
    M.Write(Client, Solenoid_Tag, Sol_Start)
    
def Window_Frame_Mover(Client,Tag1, Tag2, Location1, Location2):
    return

In [16]:
from datetime import datetime
Sol_Start = 1.12349801234098
exposure = 1.3
gain = 480.12341234512345
start_time =  datetime.today().strftime('%y%m%d_%H%M%S')
filename = "{}_{:.3f}A.bmp".format(start_time, Sol_Start)
print(filename)

200916_151806_1.123A.bmp


### This part is going to be difficult
#### We are now going to figure out how to move Window Frames 1 and 2

Something that I think will help is if the Solnoid check could somehow look how far the bean moved if at all. Maybe don't make this pass or fail, instead show a quantitative value with the potential of having a pass condition in the future.

#### WF Mover algorithm
    - Do a Solenoid check
    - Perturb in one direction by a substantial amount by doing multiple solenoid checks, maybe 3-10 at a time
    - Look at the trend but perturb the other way anyway.
    - Look at the overall trend, reduce the resolution, and try to perturb in smaller steps, but change the axis
    - We could either make this intelligent or we could make it stupid. Either way it's an optimization problem.
    - Do a solenoid check

In [6]:
# Code goes here 
#
#
#
#
#
#

## Once this is optimized and/or passes:::
#### Move on to the Sol 2 Angle check, this is entirely different as this is a pass/fail condition
    - Scan Solenoid 2 slowly and methodically, comparing to the first angle
    - Once Solenoid 2 is within 41.5 degrees, take all possible points where this is the case
    - Go to the closest value to 41.5 degrees

In [7]:
# Code goes here
#
#
#
#
#
#

## Scan Solenoid 1 until smallest
    - Scan solenoid 1 until it is the smallest

In [8]:
# Code goes here
#
#
#
#
#
#

## Calculate the optimal setting of Solenoid 3 here, then ramp it to that value

In [9]:
Solenoid_3_Tag = Tags.Sol3

In [49]:
import numpy as np
import scipy as sp
import scipy.linalg as linalg
from sympy import *
init_printing(use_unicode=True)

Phi = 'k-lhat'
Phi = 0
k = np.pi

initial = np.matrix([['x_i'],['xprime_i'],['y_i'],['yprime_i']])
final = np.matrix([['x_f'],['xprime_f'],['y_f'],['yprime_f']])
M = np.matrix([['cos(Phi)^2', '1/(2*k)*sin(2*Phi)', '1/2*sin(Phi)', '1/(k)*sin(Phi)^2'],
              ['-k/2*sin(2*Phi)', 'cos(Phi)^2', '-k*sin(Phi)^2', '1/2*sin(2*Phi)'],
              ['-1/2*sin(2*Phi)','(-1/k)*sin(Phi)^2','cos(Phi)^2','(1/(2*k))*sin(2*Phi)'],
              ['k*sin(Phi)^2','(-1/2)*sin(2*Phi)','(-k/2)*sin(2*Phi)','cos(Phi)^2']])
Matrix(np.matrix([[0],[0],[0],[0]]))

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦